# C4_200M (edits 00000) → sentence Builder


In [ ]:
%%bash
pip -q install huggingface_hub absl-py datasets >/dev/null
apt -y install git >/dev/null
echo 'Installed: huggingface_hub, absl-py, datasets, git'

In [ ]:
from pathlib import Path
import os
USE_DRIVE  = True
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.makedirs(DRIVE_DIR, exist_ok=True)
os.makedirs('/content/out', exist_ok=True)
os.makedirs('/content/c4_json', exist_ok=True)
print('Prepared folders.')

In [ ]:
# C4_200M 레포 클론(스크립트 사용)
import shutil
REPO = "/content/C4_200M-syn"
if not Path(REPO).exists():
    !git clone -q https://github.com/google-research-datasets/C4_200M-synthetic-dataset-for-grammatical-error-correction {REPO}
print('Repo ready:', REPO)

In [ ]:
# 경로 확인/준비
REPO = "/content/C4_200M-syn"
EDITS_PATH = "/content/drive/MyDrive/LikeLion_NLP2/Project_2/c4_200m_edits/edits.tsv-00000-of-00010"
TARGET_TSV = "/content/out/target_sentences.tsv-00000-of-00010"
PAIRS_TSV  = "/content/out/sentence_pairs.tsv-00000-of-00010"
DATASET_DIR =  "/C4/en"

# 캐글샤드
print("[CONFIG] EDITS_PATH:", EDITS_PATH)
# 만들 이름
print("[CONFIG] TARGET_TSV:", TARGET_TSV)
# 깃 클론
print("[CONFIG] REPO:", REPO)
# allenai에서 받은 json파일 디렉토리
print("[CONFIG] DATASET_DIR:", DATASET_DIR)


import os, gzip, zipfile, shutil
from pathlib import Path

os.makedirs("/content/out", exist_ok=True)

# edits가 zip/gzip이면 평문으로 풀기 (원본 파일 사용시 UTF-8 에러)
def ensure_plain_tsv(path):
    p = Path(path)
    with open(p, "rb") as f:
        sig = f.read(4)
    if sig.startswith(b"PK\x03\x04"):   # zip
        tmp = "/content/tmp_edits.tsv"
        zf = zipfile.ZipFile(path)
        members = [m for m in zf.infolist() if not m.is_dir()]
        assert members, "Zip 안에 파일이 없습니다."
        with zf.open(members[0], "r") as zin, open(tmp, "wb") as fout:
            shutil.copyfileobj(zin, fout)
        return tmp
    elif sig.startswith(b"\x1f\x8b"):   # gzip
        tmp = "/content/tmp_edits.tsv"
        with gzip.open(path, "rb") as fin, open(tmp, "wb") as fout:
            shutil.copyfileobj(fin, fout)
        return tmp
    else:
        return str(p)

EDITS_FLAT = ensure_plain_tsv(EDITS_PATH)
print("[INFO] using edits:", EDITS_FLAT, "| size:", Path(EDITS_FLAT).stat().st_size)

# ★★★ 핵심: JSON 스크립트를 '위치 인자 3개'로 호출해야 함
#     <edits-tsv> <dataset_dir> <output-tsv>
# dataset_dir 는 /C4 (그 아래에 en/c4-train.00000-of-01024.json.gz ... 가 있어야 함)
!python {REPO}/c4200m_get_target_sentences_json.py "{EDITS_FLAT}" "{DATASET_DIR}" "{TARGET_TSV}"

# 생성 확인
from pathlib import Path
assert Path(TARGET_TSV).exists() and Path(TARGET_TSV).stat().st_size > 0, "target TSV 생성 실패"

# (noisy, clean) 쌍 만들기 — 위치 인자 3개
# <target-tsv> <edits-tsv> <output-pairs-tsv>
!python {REPO}/c4200m_make_sentence_pairs.py "{TARGET_TSV}" "{EDITS_FLAT}" "{PAIRS_TSV}"

# 결과 요약
!wc -l {PAIRS_TSV} | sed -e 's/^/lines: /'
!head -n 3 {PAIRS_TSV}


[INFO] using edits: /content/tmp_edits.tsv | size: 2992601947
Loading C4_200M target sentence hashes from '/content/tmp_edits.tsv'...
Searching for 19938472 target sentences in the C4 dataset...
-- 0 C4 examples done, 19938472 sentences still to be found
-- 100000 C4 examples done, 19933449 sentences still to be found
-- 200000 C4 examples done, 19928311 sentences still to be found
-- 300000 C4 examples done, 19923225 sentences still to be found
-- 400000 C4 examples done, 19918291 sentences still to be found
-- 500000 C4 examples done, 19913242 sentences still to be found
-- 600000 C4 examples done, 19908241 sentences still to be found
-- 700000 C4 examples done, 19903385 sentences still to be found
-- 800000 C4 examples done, 19898350 sentences still to be found
-- 900000 C4 examples done, 19893495 sentences still to be found
-- 1000000 C4 examples done, 19888455 sentences still to be found
-- 1100000 C4 examples done, 19883441 sentences still to be found
-- 1200000 C4 examples done,